In [55]:
import pandas as pd
import tensorflow as tf
import numpy as np

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

labels = ["Lucky Star 1", "Lucky Star 2", "Ball 1", "Ball 2", "Ball 3", "Ball 4", "Ball 5"]
column_labels = ["Lucky Star 1", "Lucky Star 2", "Ball 1", "Ball 2", "Ball 3", "Ball 4", "Ball 5", "Ball Delta", "Ball Y Delta", "LS Delta",
    "Ball 1 Repeat Last 5", "Ball 2 Repeat Last 5", "Ball 3 Repeat Last 5", "Ball 4 Repeat Last 5", "Ball 5 Repeat Last 5", "Lucky Star 1 Repeat Last 5", "Lucky Star 2 Repeat Last 5",
    "Ball 1 Repeat Last 10", "Ball 2 Repeat Last 10", "Ball 3 Repeat Last 10", "Ball 4 Repeat Last 10", "Ball 5 Repeat Last 10", "Lucky Star 1 Repeat Last 10", "Lucky Star 2 Repeat Last 10",
    "Ball 1 Repeat Last 20", "Ball 2 Repeat Last 20", "Ball 3 Repeat Last 20", "Ball 4 Repeat Last 20", "Ball 5 Repeat Last 20", "Lucky Star 1 Repeat Last 20", "Lucky Star 2 Repeat Last 20", "Ball Y Delta 2"
]
column_indices = {name: i for i, name in enumerate(column_labels)}

Num GPUs Available:  0


In [56]:
df = pd.read_csv('euromillions-dataset.csv')
df["Ball Y Delta 2"] = df["Ball Y Delta"]
df

,Lucky Star 1,Lucky Star 2,Ball 1,Ball 2,Ball 3,Ball 4,Ball 5,Ball Delta,Ball Y Delta,LS Delta,...,Lucky Star 1 Repeat Last 10,Lucky Star 2 Repeat Last 10,Ball 1 Repeat Last 20,Ball 2 Repeat Last 20,Ball 3 Repeat Last 20,Ball 4 Repeat Last 20,Ball 5 Repeat Last 20,Lucky Star 1 Repeat Last 20,Lucky Star 2 Repeat Last 20,Ball Y Delta 2
0,1,9,3,19,29,35,37,4.974937,3.768289,8,...,0,3,1,3,4,1,1,2,6,3.768289
1,8,12,19,24,26,28,33,1.903943,2.986637,4,...,2,2,2,2,0,1,4,4,4,2.986637
2,4,8,12,22,27,33,45,4.366062,5.803447,4,...,1,2,2,1,1,3,2,1,3,5.803447
3,11,12,6,10,16,23,24,2.524876,5.772348,1,...,1,2,0,3,0,2,1,1,4,5.772348
4,6,9,10,14,21,33,50,5.578978,6.560488,3,...,3,3,2,1,1,2,4,4,6,6.560488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,6,9,1,19,36,38,49,6.791539,2.814249,3,...,0,1,0,1,1,0,0,0,1,2.814249
298,9,10,10,25,29,34,45,4.918079,3.143247,1,...,0,0,0,0,0,0,0,0,0,3.143247
299,2,4,6,16,18,39,47,6.169481,3.831449,2,...,1,0,0,0,0,1,0,1,0,3.831449
300,7,12,5,14,35,36,39,5.766281,4.422669,5,...,0,0,0,0,0,0,0,0,0,4.422669


In [57]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# --- 1. Data Preparation ---

def create_sequences(data, seq_length, pred_cols):
    """
    Creates sequences and corresponding labels from a Pandas DataFrame.

    Args:
        data: Pandas DataFrame.
        seq_length: Length of the input sequence.
        pred_cols: Number of columns to predict (starting from the first column).

    Returns:
        X: Input sequences (NumPy array).
        y: Target values (NumPy array).
    """
    xs = []
    ys = []
    for i in range(len(data) - seq_length):
        x = data[i:(i + seq_length)].values  # Sequence of 'seq_length' rows
        y = data.iloc[i + seq_length, :pred_cols].values  # Predict next row's first 'pred_cols' columns
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)


# --- 2. Load and Preprocess Data ---

# Parameters
seq_length = 7  # Number of past rows to look at
pred_cols = 7  # Number of columns to predict
all_cols = 32

# Scale the data
scaler = MinMaxScaler()
df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

# Create sequences
X, y = create_sequences(df, seq_length, pred_cols)

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [58]:
X_train[0]

array([[0.4       , 0.4       , 0.11764706, 0.22222222, 0.35714286,
        0.43243243, 0.39393939, 0.27947171, 0.35435474, 0.        ,
        0.        , 0.33333333, 0.25      , 0.33333333, 0.        ,
        0.        , 0.        , 0.        , 0.2       , 0.4       ,
        0.5       , 0.2       , 0.        , 0.16666667, 0.        ,
        0.5       , 0.42857143, 0.5       , 0.28571429, 0.22222222,
        0.1       , 0.35435474],
       [0.8       , 1.        , 0.20588235, 0.47222222, 0.66666667,
        0.81081081, 0.75757576, 0.50323628, 0.16381012, 0.2       ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.33333333, 1.        , 0.25      , 0.2       , 0.        ,
        0.        , 0.        , 0.6       , 1.        , 0.16666667,
        0.33333333, 0.        , 0.16666667, 0.14285714, 0.77777778,
        0.9       , 0.16381012],
       [0.2       , 1.        , 0.38235294, 0.58333333, 0.83333333,
        1.        , 1.        , 0.55078026, 0.1670

In [65]:
class TemperatureLayer(tf.keras.layers.Layer):
  def __init__(self, temperature=1.0, **kwargs):
    super(TemperatureLayer, self).__init__(**kwargs)
    self.temperature = tf.constant(temperature)  # Initialize with a default temperature

  def call(self, inputs):
    # Introduce Gaussian noise scaled by the temperature
    noise = tf.random.normal(shape=tf.shape(inputs), mean=0.0, stddev=self.temperature)
    return inputs + noise

  def get_config(self):
    config = super().get_config()
    config.update({"temperature": self.temperature})
    return config

In [88]:
# --- 3. Model Definition ---

lstm_model = tf.keras.models.Sequential([
    # First 1D Convolutional layer
    tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(seq_length, all_cols)),
    tf.keras.layers.MaxPooling1D(pool_size=2),

    # LSTM layer
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.Dropout(0.2),

    # Reshape for Dense output
    tf.keras.layers.Reshape((64, 2)),  # Reshape to add a time dimension

    # Second 1D Convolutional layer
    tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),

    # Flatten for Dense output
    tf.keras.layers.Flatten(),

    # Output layer
    tf.keras.layers.Dense(pred_cols),
    TemperatureLayer(temperature=0.1) # Add the custom layer after the Dense layer, start with a temperature around 1.5 or 2.
])


In [89]:

# --- 4. Loss Function with Emphasis ---

def weighted_mse(y_true, y_pred):
    """
    Custom loss function that emphasizes the first 'pred_cols' columns.

    Args:
        y_true: True values.
        y_pred: Predicted values.

    Returns:
        Weighted mean squared error.
    """
    # Calculate the squared error for all columns
    squared_error = tf.square(y_true - y_pred)

    # Create a tensor of weights with higher weight for the first 'pred_cols'
    weights = tf.concat([tf.ones(pred_cols) * 7, tf.ones(all_cols - pred_cols)], axis=0)

    # Apply the weights to the squared error. Only apply to the relevant columns
    weighted_squared_error = squared_error * weights[:pred_cols]

    # Calculate the mean of the weighted squared error
    return tf.reduce_mean(weighted_squared_error)


# --- 5. Compile the Model ---

lstm_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                   loss=weighted_mse)  # Use the custom weighted loss

# --- 6. Callbacks ---
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)


In [90]:
# --- 7. Train the Model ---

history = lstm_model.fit(X_train, y_train,
                         epochs=100,
                         batch_size=32,
                         validation_data=(X_val, y_val),
                         callbacks=[early_stopping, reduce_lr])

# --- 8. Evaluate and Make Predictions ---
# (Example - adapt as needed)

val_loss = lstm_model.evaluate(X_val, y_val)
print(f"Validation Loss: {val_loss}")

# During inference, you can adjust the temperature of the TemperatureLayer
# For example, to set a new temperature:
lstm_model.layers[-1].temperature = tf.constant(0.1)  # Increase temperature for more randomness

# Make predictions on the validation set
predictions = lstm_model.predict(X_val)

# Inverse transform to get predictions in the original scale
# (If you scaled your data)
predictions = scaler.inverse_transform(np.concatenate((predictions, np.zeros((predictions.shape[0], all_cols - pred_cols))), axis=1))[:, :pred_cols]
y_val_inv = scaler.inverse_transform(np.concatenate((y_val, np.zeros((y_val.shape[0], all_cols - pred_cols))), axis=1))[:, :pred_cols]

# Now you can compare 'predictions' with 'y_val_inv' to see how well your model did

Epoch 1/100
8/8 [==============================] - 2s 47ms/step - loss: 1.8546 - val_loss: 0.9096 - lr: 0.0010
Epoch 2/100
8/8 [==============================] - 0s 6ms/step - loss: 0.6729 - val_loss: 0.4833 - lr: 0.0010
Epoch 3/100
8/8 [==============================] - 0s 6ms/step - loss: 0.5015 - val_loss: 0.5879 - lr: 0.0010
Epoch 4/100
8/8 [==============================] - 0s 6ms/step - loss: 0.5101 - val_loss: 0.4818 - lr: 0.0010
Epoch 5/100
8/8 [==============================] - 0s 5ms/step - loss: 0.4448 - val_loss: 0.4442 - lr: 0.0010
Epoch 6/100
8/8 [==============================] - 0s 5ms/step - loss: 0.4410 - val_loss: 0.4818 - lr: 0.0010
Epoch 7/100
8/8 [==============================] - 0s 6ms/step - loss: 0.4364 - val_loss: 0.4467 - lr: 0.0010
Epoch 8/100
8/8 [==============================] - 0s 6ms/step - loss: 0.4448 - val_loss: 0.4152 - lr: 0.0010
Epoch 9/100
8/8 [==============================] - 0s 6ms/step - loss: 0.4430 - val_loss: 0.4619 - lr: 0.0010
Epoch 10/

In [91]:
predictions

array([[ 2.93146437,  8.25868738,  4.37196034, 14.14086676, 18.48098397,
        29.38758999, 31.89386168],
       [ 5.30246174,  6.22254384,  8.74558848, 17.06134069, 19.13599479,
        32.36400175, 38.31570548],
       [ 4.67611378,  7.97101927,  5.25107357, 17.14832902, 24.13661909,
        27.75018299, 46.9565922 ],
       [ 3.88116336,  5.95975888,  5.56420767, 16.55376375, 22.84265924,
        36.0643158 , 42.38426572],
       [ 4.61819148,  8.9338572 , 10.44283378, 16.82180226, 29.18997991,
        36.44992507, 38.47422814],
       [ 2.2683554 ,  7.79835296, 12.28530425, 13.53137231, 16.61438751,
        29.48672146, 39.23588949],
       [ 1.83524093,  8.35897815, 10.74588019, 15.60927641, 22.34978956,
        36.7488544 , 42.63425368],
       [ 4.5514918 ,  7.20552516, 11.34045607, 18.55007076, 27.04890871,
        31.78727275, 37.03148562],
       [ 2.81338578,  7.47785819,  6.11215955, 15.58328903, 16.24168205,
        23.36611766, 38.50673795],
       [ 3.89540559,  9.2540

In [92]:
y_val_inv

array([[ 4., 10.,  2.,  7., 21., 31., 45.],
       [ 9., 11.,  7., 26., 34., 41., 44.],
       [ 3.,  7., 23., 31., 37., 42., 48.],
       [ 5.,  6.,  3.,  4.,  9., 12., 20.],
       [ 4.,  8.,  3., 11., 17., 45., 48.],
       [ 1.,  3.,  2., 33., 35., 42., 48.],
       [ 7.,  8.,  2., 32., 35., 36., 39.],
       [ 2.,  8.,  1.,  2., 12., 19., 26.],
       [10., 12.,  6., 20., 40., 42., 43.],
       [ 3., 12., 14., 23., 39., 48., 50.],
       [ 4.,  9.,  2., 11., 28., 42., 48.],
       [ 6., 11., 12., 21., 35., 39., 45.],
       [ 7.,  9.,  2.,  8., 17., 28., 35.],
       [ 1.,  8.,  3.,  5., 32., 33., 50.],
       [ 4.,  9.,  3., 25., 27., 28., 29.],
       [ 6.,  8.,  2.,  7., 34., 35., 46.],
       [ 1., 11., 11., 13., 29., 31., 47.],
       [ 5., 11., 18., 19., 21., 27., 47.],
       [ 4.,  5.,  7., 10., 22., 29., 44.],
       [10., 12.,  1.,  7., 24., 41., 48.],
       [ 6.,  9.,  7., 32., 33., 34., 38.],
       [ 3.,  7.,  3.,  8., 18., 34., 49.],
       [ 1.,  7., 17., 22., 23.,

In [98]:
# Now to predict just the next lotto numbers based on the last 7

# Get the last sequence from X_val
last_sequence = X_val[-1].reshape(1, seq_length, all_cols)  # Reshape to add a batch dimension

# Make a prediction using the last sequence
prediction = lstm_model.predict(last_sequence)

# Inverse transform the prediction (if you scaled your data)
prediction = scaler.inverse_transform(np.concatenate((prediction, np.zeros((prediction.shape[0], all_cols - pred_cols))), axis=1))[:, :pred_cols]

prediction

1/1 [==============================] - 0s 12ms/step


array([[ 1.17265052,  7.20402193,  6.94183642, 13.15710652, 25.38682461,
        34.24314398, 39.74264944]])